In [30]:
# !pip install pytimeparse

In [31]:
# !pip install mealpy

In [32]:
import numpy as np
import matplotlib.pyplot as plt
import csv
import datetime

## Weekly

In [33]:
days = 7
int_columns = [0, 1, 2, 3]
numeric_columns = [5, 6, 7]
time_columns = [4]

file_name = 'pairwise_deals_train_' + str(days) + '.csv'
pairwise_deals_train = list()
with open(file_name) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            column_names = row
            line_count += 1
        else:
            for int_element in int_columns:
                row[int_element] = int(row[int_element])
            for numeric_element in numeric_columns:
                row[numeric_element] = float(row[numeric_element])
            pairwise_deals_train.append(row)
            line_count += 1
    print(f'Processed {line_count} lines.')


file_name = 'pairwise_deals_test_' + str(days) + '.csv'
pairwise_deals_test = list()
with open(file_name) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            column_names = row
            line_count += 1
        else:
            for int_element in int_columns:
                row[int_element] = int(row[int_element])
            for numeric_element in numeric_columns:
                row[numeric_element] = float(row[numeric_element])
            pairwise_deals_test.append(row)
            line_count += 1
    print(f'Processed {line_count} lines.')

Column names are bird_index_1, bird_index_2, bird_id_1, bird_id_2, time_between_deals, log_price_ratio, price_1, price_2
Processed 474297 lines.
Column names are bird_index_1, bird_index_2, bird_id_1, bird_id_2, time_between_deals, log_price_ratio, price_1, price_2
Processed 41661 lines.


In [34]:
from pytimeparse.timeparse import timeparse

time_delta_seconds = days * 24 * 60 * 60

print(abs(timeparse(pairwise_deals_train[100][4])) / time_delta_seconds)
print(pairwise_deals_train[100][4])

0.4604861111111111
-3 days, 5:21:42


In [35]:
column_names = ['bird_id', 'theme', 'head', 'eyes', 'body', 'tail', 'wingLeft', 'wingRight', 'feet', 'beak',
                'trait_score', 'trait_rank', 'set_score', 'set_rank', 'edition_score', 'edition_rank', 
                'weighted_score', 'weighted_rank', 'birds_weighted_normalized_scores', 'day_trade_score', 'week_trade_score']

birds_dataset_with_ranks = list()
int_columns = [0, 11, 13, 15, 17]
numeric_columns = [10, 12, 14, 16] # , 18, 19, 20]
with open('birds_dataset_with_ranks.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            column_names = row
            line_count += 1
        else:
            for int_element in int_columns:
                row[int_element] = int(row[int_element])
            for numeric_element in numeric_columns:
                row[numeric_element] = float(row[numeric_element])
            birds_dataset_with_ranks.append(row)
            line_count += 1
    print(f'Processed {line_count} lines.')

Column names are bird_id, theme, head, eyes, body, tail, wingLeft, wingRight, feet, beak, trait_score, trait_rank, set_score, set_rank, edition_score, edition_rank, weighted_score, weighted_rank
Processed 8479 lines.


In [36]:
column_indexes = [10, 12, 14, 16]
scores = [[x[i] for i in column_indexes] for x in birds_dataset_with_ranks]
scores = np.array(scores)
print(len(scores), len(scores[0]))

8478 4


In [37]:
def kernel_epanechnikov(time_deltas, days_width=7):
    time_delta_seconds = days_width * 24 * 60 * 60
    kernel_values = np.zeros(len(time_deltas))
    for index in range(len(time_deltas)):
        kernel_values[index] = max(1 - (abs(timeparse(time_deltas[index])) / time_delta_seconds) ** 2, 0)
    return kernel_values

time_deltas_train = [x[4] for x in pairwise_deals_train]

kernal_values_train = kernel_epanechnikov(time_deltas_train)
kernal_values_train = np.array(kernal_values_train)

time_deltas_test = [x[4] for x in pairwise_deals_test]

kernal_values_test = kernel_epanechnikov(time_deltas_test)
kernal_values_test = np.array(kernal_values_test)

In [38]:
relatile_log_price_train = [np.log(x[-2]/x[-1]) for x in pairwise_deals_train]
relatile_log_price_train = np.array(relatile_log_price_train)

relatile_log_price_test = [np.log(x[-2]/x[-1]) for x in pairwise_deals_test]
relatile_log_price_test = np.array(relatile_log_price_test)

In [39]:
pair_indexes_train = np.array([x[:2] for x in pairwise_deals_train])
pair_indexes_test = np.array([x[:2] for x in pairwise_deals_test])
print(pair_indexes_train[:10])
print(type(pair_indexes_train[0, 0]))

[[  3   4]
 [  3   6]
 [  3  34]
 [  3  46]
 [  3 165]
 [  3 177]
 [  3 177]
 [  3 177]
 [  3 181]
 [  3 197]]
<class 'numpy.int32'>


In [40]:
def m(x, w):
    """Weighted Mean"""
    return np.sum(x * w) / np.sum(w)

def cov(x, y, w):
    """Weighted Covariance"""
    return np.sum(w * (x - m(x, w)) * (y - m(y, w))) / np.sum(w)

def corr(x, y, w):
    """Weighted Correlation"""
    return cov(x, y, w) / np.sqrt(cov(x, x, w) * cov(y, y, w))

def get_accuracy(scores_combination, 
                 pair_indexes=pair_indexes_train, 
                 relatile_log_price=relatile_log_price_train, 
                 kernal_values=kernal_values_train):
    return corr(np.log((1 + scores_combination[pair_indexes[:, 0]]) / 
                              (1 + scores_combination[pair_indexes[:, 1]])), relatile_log_price, kernal_values)

def objective(coefficients, scores=scores[:, :3]):
    coefficients = np.append(coefficients, (1 - coefficients[0] - coefficients[1]))
    return -get_accuracy(np.matmul(scores, coefficients))

def get_accuracy_test(scores_combination):
    return get_accuracy(scores_combination, pair_indexes_test, relatile_log_price_test, kernal_values_test)

def objective_test(coefficients, scores=scores[:, :3]):
    coefficients = np.append(coefficients, (1 - coefficients[0] - coefficients[1]))
    return -get_accuracy_test(np.matmul(scores, coefficients))

In [41]:
print(np.shape(pair_indexes_train))
print(np.shape(relatile_log_price_train))

(474296, 2)
(474296,)


In [42]:
print(get_accuracy(scores[:, 0]), get_accuracy(scores[:, 1]), get_accuracy(scores[:, 2]), get_accuracy(scores[:, 3]))

print(get_accuracy_test(scores[:, 0]),
      get_accuracy_test(scores[:, 1]),
      get_accuracy_test(scores[:, 2]),
      get_accuracy_test(scores[:, 3]))

0.5645546159211253 0.37954716374697756 0.7366183863651179 0.7636413840354973
0.5775619945190855 0.39163046255649697 0.8574641366925181 0.8462000355628161


In [43]:
birds_trait_scores = np.array([x[-8] for x in birds_dataset_with_ranks])
birds_set_scores = np.array([x[-6] for x in birds_dataset_with_ranks])
birds_edition_scores = np.array([x[-4] for x in birds_dataset_with_ranks])

birds_weighted_normalized_scores = (birds_trait_scores + birds_set_scores + birds_edition_scores) / 3

print(get_accuracy(birds_weighted_normalized_scores))
print(objective([1/3, 1/3]))
print(get_accuracy_test(birds_weighted_normalized_scores))

0.7636413840354973
-0.7636413840354973
0.8462000355628161


In [44]:
from scipy.optimize import minimize, LinearConstraint

In [45]:
linear_constraint = LinearConstraint([[1, 0], [0, 1], [1, 1]], [0, 0, 0], [1, 1, 1])

x0 = [1/3, 1/3]
res = minimize(objective, x0, constraints=linear_constraint, options={'disp': True}) # method='nelder-mead',
print(res)
coefficents = res.x
coefficents = np.append(coefficents, 1 - np.sum(res.x))
print(get_accuracy_test(np.matmul(scores[:, :3], coefficents)))
print(objective_test(res.x))

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.8239566862393403
            Iterations: 7
            Function evaluations: 23
            Gradient evaluations: 7
     fun: -0.8239566862393403
     jac: array([0.00074369, 0.06660111])
 message: 'Optimization terminated successfully'
    nfev: 23
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([7.52528212e-01, 1.63523752e-17])
0.8801369485963232
-0.8801369485963232


In [46]:
print(birds_dataset_with_ranks[0])

[1, 'cyberbird', 'cyberbird', 'cyberbird', 'cyberbird', 'cyberbird', 'cyberbird', 'cyberbird', 'cyberbird', 'var2', 3.898092031672943, 4, 1.9994714877881563, 1, 14.350200592323434, 1, 6.749254703928178, 1]


In [47]:
set_size = 10
x0_points = list()
x_points = list()
y_points = list()
for iteration in range(set_size):
    print('iteration: ', iteration)
    boundaries = sorted([np.random.rand(), np.random.rand()])
    x0 = [boundaries[0], boundaries[1] - boundaries[0]]
    x0_points.append(x0)
    res = minimize(objective, x0, constraints=linear_constraint, options={'disp': True})
    x_current = res.x
    y_current = res.fun
    x_points.append(x_current)
    y_points.append(y_current)

iteration:  0
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.8239568231782186
            Iterations: 14
            Function evaluations: 46
            Gradient evaluations: 14
iteration:  1
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.8239568251398421
            Iterations: 14
            Function evaluations: 48
            Gradient evaluations: 14
iteration:  2
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.823956372324089
            Iterations: 7
            Function evaluations: 24
            Gradient evaluations: 7
iteration:  3
Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.8239568244706403
            Iterations: 13
            Function evaluations: 42
            Gradient evaluations: 13
iteration:  4
Optimization terminated successfully    (Exit mode 0)
            Current function value:

In [48]:
minimum_index = np.argmin(y_points)
print(x_points[minimum_index], -y_points[minimum_index])
optimal_weekly_x = x_points[minimum_index]

[7.52174585e-01 9.61753357e-18] 0.8239568278650474


In [49]:
1 - optimal_weekly_x[0] - optimal_weekly_x[1]

0.24782541536003355

In [50]:
'''from mealpy.bio_based import SMA
import numpy as np


def obj_function(solution):
    
    def violate(value):
        return 0 if value >= 0 else value
    
    def margine(solution):
        return 1 - solution[0] - solution[1]

    fx = objective(solution)

## Increase the punishment for g1 and g4 to boost the algorithm (You can choice any constraint instead of g1 and g4)
    fx = fx - violate(margine(solution))
  
    return fx

problem_dict1 = {
    "obj_func": obj_function,
    "lb": [0, 0],
    "ub": [1, 1],
    "minmax": "min",
    "verbose": False,
}

## Run the algorithm
model1 = SMA.BaseSMA(problem_dict1, epoch=100, pop_size=50, pr=0.03)
model1.solve()'''

'from mealpy.bio_based import SMA\nimport numpy as np\n\n\ndef obj_function(solution):\n    \n    def violate(value):\n        return 0 if value >= 0 else value\n    \n    def margine(solution):\n        return 1 - solution[0] - solution[1]\n\n    fx = objective(solution)\n\n## Increase the punishment for g1 and g4 to boost the algorithm (You can choice any constraint instead of g1 and g4)\n    fx = fx - violate(margine(solution))\n  \n    return fx\n\nproblem_dict1 = {\n    "obj_func": obj_function,\n    "lb": [0, 0],\n    "ub": [1, 1],\n    "minmax": "min",\n    "verbose": False,\n}\n\n## Run the algorithm\nmodel1 = SMA.BaseSMA(problem_dict1, epoch=100, pop_size=50, pr=0.03)\nmodel1.solve()'

In [51]:
'''print(model1.solution[0])
print(objective(model1.solution[0]))
print(objective_test(model1.solution[0]))'''

'print(model1.solution[0])\nprint(objective(model1.solution[0]))\nprint(objective_test(model1.solution[0]))'

## Save

In [52]:
def score_from_coefficients(coefficients):
    return birds_trait_scores * coefficients[0] + birds_set_scores * coefficients[1] + birds_edition_scores * (1 - coefficients[0] - coefficients[1])

In [53]:
birds_trait_scores = np.array([x[-8] for x in birds_dataset_with_ranks])
birds_set_scores = np.array([x[-6] for x in birds_dataset_with_ranks])
birds_edition_scores = np.array([x[-4] for x in birds_dataset_with_ranks])

In [55]:
optimal_weekly_x = [7.52172156e-01, 2.55288896e-17]
print([optimal_weekly_x[0], optimal_weekly_x[1], 1 - optimal_weekly_x[0] - optimal_weekly_x[1]])
# [0.76, 0, 0.24]

[0.752172156, 2.55288896e-17, 0.247827844]


In [56]:
birds_weighted_normalized_scores = (birds_trait_scores / np.mean(birds_trait_scores)) + (
                                    birds_set_scores / np.mean(birds_set_scores)) + (
                                    birds_edition_scores / np.mean(birds_edition_scores))

week_trade_score = score_from_coefficients(optimal_weekly_x) 


In [57]:
nfts_number = len(birds_dataset_with_ranks)
birds_dataset_with_ranks = np.hstack((birds_dataset_with_ranks, 
                                      np.reshape(birds_weighted_normalized_scores, (nfts_number, 1)),
                                      np.reshape(week_trade_score, (nfts_number, 1))
                                     ))

In [58]:
column_names = ['bird_id', 'theme', 'head', 'eyes', 'body', 'tail', 'wingLeft', 'wingRight', 'feet', 'beak',
                'trait_score', 'trait_rank', 'set_score', 'set_rank', 'edition_score', 'edition_rank', 
                'weighted_score', 'weighted_rank', 
                'birds_weighted_normalized_scores', 'week_trade_score']

with open('birds_dataset_with_ranks_fitted.csv', 'w', newline='') as birds_file:
    bird_writer = csv.writer(birds_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    bird_writer.writerow(column_names)
    for row_index in range(len(birds_dataset_with_ranks)):
        bird_writer.writerow(birds_dataset_with_ranks[row_index])

birds_dataset_with_ranks = list()
int_columns = [0, -1-2, -3-2, -5-2, -7-2]
numeric_columns = [-2-2, -4-2, -6-2, -8-2, -2, -1]
with open('birds_dataset_with_ranks_fitted.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        if line_count == 0:
            print(f'Column names are {", ".join(row)}')
            column_names = row
            line_count += 1
        else:
            for int_element in int_columns:
                row[int_element] = int(row[int_element])
            for numeric_element in numeric_columns:
                row[numeric_element] = float(row[numeric_element])
            birds_dataset_with_ranks.append(row)
            line_count += 1
    print(f'Processed {line_count} lines.')

Column names are bird_id, theme, head, eyes, body, tail, wingLeft, wingRight, feet, beak, trait_score, trait_rank, set_score, set_rank, edition_score, edition_rank, weighted_score, weighted_rank, birds_weighted_normalized_scores, week_trade_score
Processed 8479 lines.


## Visualize ranks

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

nfts_number = len(birds_dataset_with_ranks)

rank_indexes = [-4, -2, -1]
table_names = ['1/3', 'normalize and 1/3', 'fit on weekly data']
for current_rank_index in range(len(rank_indexes)):

    bird_indexes = np.array([x[0] for x in birds_dataset_with_ranks])
    bird_scores = np.array([x[rank_indexes[current_rank_index]] for x in birds_dataset_with_ranks])
    bird_ranks = sorted(range(len(bird_scores)), key=lambda k: -bird_scores[k])
    bird_ranks = [x + 1 for x in bird_ranks]
    
    borders = [0, 10, 100, 1000, 10000]

    birds_scores = np.zeros((nfts_number,))
    edition_trait = np.zeros((nfts_number,))
    for current_border in borders:
        edition_trait = edition_trait + (bird_indexes < current_border)
    edition_trait = [int(x) for x in edition_trait]
    edition_collections_number = len(np.unique(edition_trait))
    data = np.zeros((edition_collections_number, edition_collections_number))
    for row in range(edition_collections_number):
        for column in range(edition_collections_number):
            for bird_index in range(len(bird_indexes)):
                if ((bird_indexes[bird_index] >= borders[row]) and 
                    (bird_indexes[bird_index] < borders[row + 1]) and
                    (bird_ranks[bird_index] >= borders[column]) and
                    (bird_ranks[bird_index] < borders[column + 1])
                   ):
                    data[row, column] = data[row, column] + 1
    columns = ('Among 1-9', '10-99', '100-999', '1000-9999')
    rows = ['Super Founder', 'Founder', 'Rare', 'Limited Edition']

    # Get some pastel shades for the colors
    colors = ['orange', 'blueviolet', 'blue', 'limegreen']# plt.cm.BuPu(np.linspace(0, 0.5, len(rows)))
    n_rows = len(data)

    import numpy as np
    import pandas as pd
    import matplotlib.pyplot as plt
    

    fig, ax = plt.subplots()

    # hide axes
    fig.patch.set_visible(False)
    ax.axis('off')
    ax.axis('tight')

    df = pd.DataFrame(data.transpose(), columns=columns, index=rows).astype(int)
    
    

    ax.table(cellText=df.values, rowLabels=rows, rowColours=colors, colLabels=df.columns, colColours=colors, loc='center')
    ax.set_title(table_names[current_rank_index])
    fig.tight_layout()

    plt.show()